# Enefit Prosumer Energy Challenge

## Introduction
The aim of this competition is to develop a predictive model that can precisely anticipate the energy usage patterns of prosumers, thereby helping to minimize the costs that arise from energy imbalances.

For comprehensive details on the competition, visit the [Enefit - Predict Energy Behavior of Prosumers | Kaggle](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/overview) page.

Also, check out the associated GitHub repository for resources and updates: [jackren0000/Enefit - Predict Energy Behavior of Prosumers](https://github.com/jackren0000/Enefit---Predict-Energy-Behavior-of-Prosumers).

In [1]:
#### import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# date and time utilities from the standard library
from datetime import datetime, date, timedelta
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
# library to suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# custom library 
# import enefit

In [2]:
#### update data directory path
DATA_DIR = '/kaggle/input/predict-energy-behavior-of-prosumers'

In [3]:
#### get the full path of all files
# walk through the every directory inside DATA_DIR 
for directory, subdirectories, files in os.walk(DATA_DIR):
    # within each directory, iterate over every filename
    for file in files:
        # join the directory path and filename
        # by using os.path.join(), you make your code portable across different operating systems
        print(os.path.join(directory, file))

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [4]:
#### read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
gas_df = pd.read_csv(os.path.join(DATA_DIR, "gas_prices.csv"))
electricity_df = pd.read_csv(os.path.join(DATA_DIR, "electricity_prices.csv"))
client_df = pd.read_csv(os.path.join(DATA_DIR, "client.csv"))
fw_df = pd.read_csv(os.path.join(DATA_DIR, "forecast_weather.csv"))
hw_df = pd.read_csv(os.path.join(DATA_DIR, "historical_weather.csv"))

# read a file from a different directory
# see how this data is generated: https://www.kaggle.com/jackren000/mapping-locations-and-county-code/edit
locations = pd.read_csv("/kaggle/input/county-lon-lats/county_lon_lats.csv")

### Data Preprocessing and Exploration

#### 1. train.csv

In [5]:
train.head(10)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
5,0,0,3,656.859,1,2021-09-01 00:00:00,0,5,2
6,0,1,0,0.000,0,2021-09-01 00:00:00,0,6,3
7,0,1,0,59.000,1,2021-09-01 00:00:00,0,7,3
8,0,1,1,0.000,0,2021-09-01 00:00:00,0,8,4
9,0,1,1,501.760,1,2021-09-01 00:00:00,0,9,4


In [6]:
print(f'The shape of the dataset is: {train.shape}')
print(f'\nThe info of the dateset is:')
print(train.info())

The shape of the dataset is: (2018352, 9)

The info of the dateset is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB
None


In [7]:
# drop missing values of 'target' column in train DataFrame
train.dropna(subset=['target'], inplace=True)

# convert datetime to UTC in train DataFrame
# the pd.to_datetime() is used to convert a string or a series of strings into pandas datetime objects
train['datetime'] = pd.to_datetime(train['datetime'], utc=True)

# add year column in train DataFrame by extracting the year part from datetime object
train['year'] = train['datetime'].dt.year

# add month column in train DataFrame
train['month'] = train['datetime'].dt.month

# add hour column in train DataFrame
train['hour'] = train['datetime'].dt.hour

# add day of week column in train DataFrame
train['dayofweek'] = train['datetime'].dt.dayofweek

# add day of year column in train DataFrame
train['dayofyear'] = train['datetime'].dt.dayofyear

In [8]:
# county from 0 to 15,
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,year,month,hour,dayofweek,dayofyear
0,0,0,1,0.713,0,2021-09-01 00:00:00+00:00,0,0,0,2021,9,0,2,244
1,0,0,1,96.590,1,2021-09-01 00:00:00+00:00,0,1,0,2021,9,0,2,244
2,0,0,2,0.000,0,2021-09-01 00:00:00+00:00,0,2,1,2021,9,0,2,244
3,0,0,2,17.314,1,2021-09-01 00:00:00+00:00,0,3,1,2021,9,0,2,244
4,0,0,3,2.904,0,2021-09-01 00:00:00+00:00,0,4,2,2021,9,0,2,244


In [9]:
# show the range of the dates
train['datetime'].unique()

<DatetimeArray>
['2021-09-01 00:00:00+00:00', '2021-09-01 01:00:00+00:00',
 '2021-09-01 02:00:00+00:00', '2021-09-01 03:00:00+00:00',
 '2021-09-01 04:00:00+00:00', '2021-09-01 05:00:00+00:00',
 '2021-09-01 06:00:00+00:00', '2021-09-01 07:00:00+00:00',
 '2021-09-01 08:00:00+00:00', '2021-09-01 09:00:00+00:00',
 ...
 '2023-05-31 14:00:00+00:00', '2023-05-31 15:00:00+00:00',
 '2023-05-31 16:00:00+00:00', '2023-05-31 17:00:00+00:00',
 '2023-05-31 18:00:00+00:00', '2023-05-31 19:00:00+00:00',
 '2023-05-31 20:00:00+00:00', '2023-05-31 21:00:00+00:00',
 '2023-05-31 22:00:00+00:00', '2023-05-31 23:00:00+00:00']
Length: 15308, dtype: datetime64[ns, UTC]

#### 2. electricity_prices.csv

In [10]:
electricity_df.head()

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1


In [11]:
################## electricity
# rename 'forecast_date' column to 'datetime' for consistency before merging
electricity_df = electricity_df.rename(columns={'forecast_date': 'datetime'}) 

# convert datetime to UTC
electricity_df['datetime'] = pd.to_datetime(electricity_df['datetime'], utc=True)

# add hour column
electricity_df['hour'] = electricity_df['datetime'].dt.hour
electricity_df.head()

,datetime,euros_per_mwh,origin_date,data_block_id,hour
0,2021-09-01 00:00:00+00:00,92.51,2021-08-31 00:00:00,1,0
1,2021-09-01 01:00:00+00:00,88.90,2021-08-31 01:00:00,1,1
2,2021-09-01 02:00:00+00:00,87.35,2021-08-31 02:00:00,1,2
3,2021-09-01 03:00:00+00:00,86.88,2021-08-31 03:00:00,1,3
4,2021-09-01 04:00:00+00:00,88.43,2021-08-31 04:00:00,1,4


In [12]:
# show the range of the date
electricity_df['datetime']

0       2021-09-01 00:00:00+00:00
1       2021-09-01 01:00:00+00:00
2       2021-09-01 02:00:00+00:00
3       2021-09-01 03:00:00+00:00
4       2021-09-01 04:00:00+00:00
                   ...           
15281   2023-05-30 19:00:00+00:00
15282   2023-05-30 20:00:00+00:00
15283   2023-05-30 21:00:00+00:00
15284   2023-05-30 22:00:00+00:00
15285   2023-05-30 23:00:00+00:00
Name: datetime, Length: 15286, dtype: datetime64[ns, UTC]

#### 3. forecast_weather.csv

In [13]:
fw_df.head(1000)

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 02:00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 03:00:00,0.000000,0.000000,0.0,0.0
1,57.6,22.2,2021-09-01 02:00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 03:00:00,0.000000,0.000000,0.0,0.0
2,57.6,22.7,2021-09-01 02:00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 03:00:00,0.000000,0.000000,0.0,0.0
3,57.6,23.2,2021-09-01 02:00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 03:00:00,0.000000,0.000000,0.0,0.0
4,57.6,23.7,2021-09-01 02:00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 03:00:00,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,59.7,22.2,2021-09-01 02:00:00,9,13.546960,8.249292,0.886169,0.000290,0.014877,0.888644,5.167040,-9.134179,1,2021-09-01 11:00:00,498.594306,455.286667,0.0,0.0
996,59.7,22.7,2021-09-01 02:00:00,9,13.764124,8.004541,0.472031,0.022095,0.017609,0.485431,5.675829,-9.080102,1,2021-09-01 11:00:00,771.980972,526.042222,0.0,0.0
997,59.7,23.2,2021-09-01 02:00:00,9,13.820520,7.535913,0.123230,0.000092,0.013428,0.134753,5.191454,-8.428979,1,2021-09-01 11:00:00,831.607639,541.615556,0.0,0.0
998,59.7,23.7,2021-09-01 02:00:00,9,13.826440,7.624048,0.000137,0.000000,0.001434,0.001574,5.121874,-8.154809,1,2021-09-01 11:00:00,832.638750,543.624444,0.0,0.0


In [14]:
# show the range of the date for the first 10000 samples
# notice: the start time is not the same as other datasets
fw_df['forecast_datetime'][:10000].unique() 

array(['2021-09-01 03:00:00', '2021-09-01 04:00:00',
       '2021-09-01 05:00:00', '2021-09-01 06:00:00',
       '2021-09-01 07:00:00', '2021-09-01 08:00:00',
       '2021-09-01 09:00:00', '2021-09-01 10:00:00',
       '2021-09-01 11:00:00', '2021-09-01 12:00:00',
       '2021-09-01 13:00:00', '2021-09-01 14:00:00',
       '2021-09-01 15:00:00', '2021-09-01 16:00:00',
       '2021-09-01 17:00:00', '2021-09-01 18:00:00',
       '2021-09-01 19:00:00', '2021-09-01 20:00:00',
       '2021-09-01 21:00:00', '2021-09-01 22:00:00',
       '2021-09-01 23:00:00', '2021-09-02 00:00:00',
       '2021-09-02 01:00:00', '2021-09-02 02:00:00',
       '2021-09-02 03:00:00', '2021-09-02 04:00:00',
       '2021-09-02 05:00:00', '2021-09-02 06:00:00',
       '2021-09-02 07:00:00', '2021-09-02 08:00:00',
       '2021-09-02 09:00:00', '2021-09-02 10:00:00',
       '2021-09-02 11:00:00', '2021-09-02 12:00:00',
       '2021-09-02 13:00:00', '2021-09-02 14:00:00',
       '2021-09-02 15:00:00', '2021-09-02 16:0

In [15]:
fw_df['origin_datetime'].unique()

array(['2021-09-01 02:00:00', '2021-09-02 02:00:00',
       '2021-09-03 02:00:00', '2021-09-04 02:00:00',
       '2021-09-05 02:00:00', '2021-09-06 02:00:00',
       '2021-09-07 02:00:00', '2021-09-08 02:00:00',
       '2021-09-09 02:00:00', '2021-09-10 02:00:00',
       '2021-09-11 02:00:00', '2021-09-12 02:00:00',
       '2021-09-13 02:00:00', '2021-09-14 02:00:00',
       '2021-09-15 02:00:00', '2021-09-16 02:00:00',
       '2021-09-17 02:00:00', '2021-09-18 02:00:00',
       '2021-09-19 02:00:00', '2021-09-20 02:00:00',
       '2021-09-21 02:00:00', '2021-09-22 02:00:00',
       '2021-09-23 02:00:00', '2021-09-24 02:00:00',
       '2021-09-25 02:00:00', '2021-09-26 02:00:00',
       '2021-09-27 02:00:00', '2021-09-28 02:00:00',
       '2021-09-29 02:00:00', '2021-09-30 02:00:00',
       '2021-10-01 02:00:00', '2021-10-02 02:00:00',
       '2021-10-03 02:00:00', '2021-10-04 02:00:00',
       '2021-10-05 02:00:00', '2021-10-06 02:00:00',
       '2021-10-07 02:00:00', '2021-10-08 02:0

In [16]:
fw_df['hours_ahead'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [17]:
locations.head(100)

,Unnamed: 0,county,longitude,latitude
0,0,0,24.2,59.1
1,1,0,25.2,59.1
2,2,0,23.7,59.4
3,3,0,24.2,59.4
4,4,0,24.7,59.4
...,...,...,...,...
70,70,15,26.7,57.6
71,71,15,27.2,57.6
72,72,15,26.7,57.9
73,73,15,27.2,57.9


In [18]:
# remove the 'Unnamed: 0' column from the 'locations' DataFrame
locations.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
################## forecast_weather dataset
################## add county code to the matching latitude and longitude

# round 'latitude' and 'longitude' to 1 decimal place
fw_df[['latitude', 'longitude']] = fw_df[['latitude', 'longitude']].astype(float).round(1)

# merge, keep all records from 'fw_df' and only add matching information from 'locations'
# add county information
fw_df = fw_df.merge(locations, how='left', on=['longitude', 'latitude'])

# drop missing values
fw_df.dropna(axis=0, inplace=True)

# convert the 'county' column to integer data type
fw_df['county'] = fw_df['county'].astype('int64')

# drop the specified columns as they are not needed for further analysis
# latitude and longitude are used for mapping the county codes
fw_df.drop(['origin_datetime', 'latitude', 'longitude', 'hours_ahead', 'data_block_id'], axis=1, inplace=True)

# rename the 'forecast_datetime' column to 'datetime' for consistency with other datasets
fw_df.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

# convert the 'datetime' column to UTC
fw_df['datetime'] = pd.to_datetime(fw_df['datetime'], utc=True)

In [20]:
fw_df.head(100)

,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,county
3,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,10
10,9.876489,9.120386,0.000000,0.025940,0.003006,0.026001,1.048755,-2.526303,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,15
11,10.815576,10.046899,0.747406,0.083191,0.000000,0.761780,1.183276,-3.873715,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,15
14,16.050439,12.355493,0.886078,0.051636,0.000092,0.889587,0.426929,-10.098569,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,10
15,14.418481,11.374536,0.855011,0.003052,0.000015,0.855301,0.902514,-6.924008,2021-09-01 03:00:00+00:00,0.0,0.0,0.0,0.000000e+00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,11.062158,9.959985,0.198441,0.999542,0.412964,0.999664,1.941915,-3.570616,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,7.450581e-07,14
149,11.487207,10.202783,0.252960,0.558014,0.314087,0.684052,1.950948,-2.862852,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,1.490116e-07,11
150,10.042505,8.659204,0.137680,0.452209,0.140244,0.547058,3.034567,-2.149473,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,0.000000e+00,11
151,9.948999,8.287012,0.001892,0.215851,0.268799,0.307404,2.855245,-1.884825,2021-09-01 04:00:00+00:00,0.0,0.0,0.0,0.000000e+00,8


In [21]:
################## forecast_weather_datetime
################## aggregate weather data into hourly averages
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = fw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = fw_df.drop(['county', 'datetime'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = fw_df.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column (add an index column)
fw_df_datetime = grouped.reset_index()

# convert hourly periods back to timestamps
# fw_df_datetime contains the average values for each hour, across all dates
fw_df_datetime['datetime'] = pd.to_datetime(fw_df_datetime['datetime'].dt.to_timestamp(), utc=True)

In [22]:
fw_df_datetime.head()

,datetime,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,2021-09-01 03:00:00+00:00,13.014066,10.531545,0.256954,0.213331,0.069662,0.438023,1.970332,-5.205125,0.000000,0.000000,0.0,0.000000e+00
1,2021-09-01 04:00:00+00:00,12.891813,10.474813,0.278619,0.178085,0.079571,0.427232,2.092311,-5.197120,0.000000,0.000000,0.0,1.151164e-06
2,2021-09-01 05:00:00+00:00,12.693667,10.327287,0.244363,0.120304,0.068174,0.368063,2.319048,-5.227491,0.000000,0.000000,0.0,8.145968e-07
3,2021-09-01 06:00:00+00:00,12.666698,10.257059,0.244026,0.108244,0.088197,0.380928,2.460921,-5.257574,57.310637,9.754364,0.0,7.553895e-07
4,2021-09-01 07:00:00+00:00,13.212046,10.436997,0.216526,0.117397,0.073299,0.344576,2.581852,-5.341152,257.277830,82.380557,0.0,1.347065e-06


In [23]:
################## forecast_weather_datetime_county
################## aggregates weather forecast data at a county level within specific hourly interval
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = fw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = fw_df.drop(['county', 'datetime'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = fw_df.groupby([hour_period, 'county'])[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column (add an index column)
fw_df_datetime_county = grouped.reset_index()

# convert hourly periods back to timestamps
# fw_df_datetime contains the average values for each county per hour, across all dates
fw_df_datetime_county['datetime'] = pd.to_datetime(fw_df_datetime_county['datetime'].dt.to_timestamp(), utc=True)

#### 4. historical_weather.csv

In [24]:
hw_df.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0


In [25]:
################## hist_weather
################## add county code to the matching latitude and longitude
# round 'latitude' and 'longitude' to 1 decimal place
hw_df[['latitude', 'longitude']] = hw_df[['latitude', 'longitude']].astype(float).round(1)

# add county information to 'hist_weather' based on matching coordinates
hw_df = hw_df.merge(locations, how='left', on=['longitude', 'latitude'])

# remove rows with any missing values
hw_df.dropna(axis=0, inplace=True)

# remove 'latitude' and 'longitude' columns as they are no longer needed
hw_df.drop(['latitude', 'longitude'], axis=1, inplace=True)

# change the 'county' column to integer datatype
hw_df['county'] = hw_df['county'].astype('int64')

# convert 'datetime' column to UTC
hw_df['datetime'] = pd.to_datetime(hw_df['datetime'], utc=True)

In [26]:
hw_df.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,data_block_id,county
3,2021-09-01 00:00:00+00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,1.0,10
10,2021-09-01 00:00:00+00:00,13.0,11.5,0.0,0.0,998.5,100,85,17,98,4.027778,354,0.0,0.0,0.0,1.0,15
11,2021-09-01 00:00:00+00:00,13.0,12.1,0.1,0.0,988.9,100,100,39,99,3.805556,360,0.0,0.0,0.0,1.0,15
14,2021-09-01 00:00:00+00:00,13.3,10.7,0.0,0.0,1016.2,51,35,0,64,7.611111,5,0.0,0.0,0.0,1.0,10
15,2021-09-01 00:00:00+00:00,14.4,11.3,0.0,0.0,1016.2,51,35,0,64,7.611111,5,0.0,0.0,0.0,1.0,10


In [27]:
################## hist_weather_datetime
################## aggregate historical weather data into hourly averages
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = hw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = hw_df.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = hw_df.groupby(hour_period)[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
hw_df_datetime = grouped.reset_index()

# convert hourly periods back to timestamps
# hw_df_datetime contains the average values for each hour, across all dates
hw_df_datetime['datetime'] = pd.to_datetime(hw_df_datetime['datetime'].dt.to_timestamp(), utc=True)
hw_df_datetime = hw_df_datetime.merge(hw_df[['datetime', 'data_block_id']], how='left', on='datetime')

In [28]:
hw_df_datetime.head()

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,data_block_id
0,2021-09-01 00:00:00+00:00,13.109333,10.790667,0.001333,0.0,1009.298667,50.986667,47.68,3.813333,23.026667,4.978148,321.333333,4.773333,3.693333,1.08,1.0
1,2021-09-01 00:00:00+00:00,13.109333,10.790667,0.001333,0.0,1009.298667,50.986667,47.68,3.813333,23.026667,4.978148,321.333333,4.773333,3.693333,1.08,1.0
2,2021-09-01 00:00:00+00:00,13.109333,10.790667,0.001333,0.0,1009.298667,50.986667,47.68,3.813333,23.026667,4.978148,321.333333,4.773333,3.693333,1.08,1.0
3,2021-09-01 00:00:00+00:00,13.109333,10.790667,0.001333,0.0,1009.298667,50.986667,47.68,3.813333,23.026667,4.978148,321.333333,4.773333,3.693333,1.08,1.0
4,2021-09-01 00:00:00+00:00,13.109333,10.790667,0.001333,0.0,1009.298667,50.986667,47.68,3.813333,23.026667,4.978148,321.333333,4.773333,3.693333,1.08,1.0


In [29]:
################## hist_weather_datetime_county
################## aggregates weather forecast data at a county level within specific hourly interval
# 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
hour_period = hw_df['datetime'].dt.to_period('h')

# 2. drop the columns we don't want to include in the mean calculation
columns_to_keep = hw_df.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

# 3. group by the hourly period and calculate the mean for the remaining columns
grouped = hw_df.groupby(['county', hour_period])[columns_to_keep].mean()

# 4. reset the index so we can have the hourly periods as a regular column
hw_df_datetime_county = grouped.reset_index()

# convert hourly periods back to timestamps
# hw_df_datetime contains the average values for each hour, across all dates
hw_df_datetime_county['datetime'] = pd.to_datetime(hw_df_datetime_county['datetime'].dt.to_timestamp(), utc=True)
hw_df_datetime_county = hw_df_datetime_county.merge(hw_df[['datetime', 'data_block_id']], how='left', on='datetime')

In [30]:
hw_df_datetime_county.head()

,county,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,data_block_id
0,0,2021-09-01 00:00:00+00:00,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325,342.0,0.0,0.0,0.0,1.0
1,0,2021-09-01 00:00:00+00:00,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325,342.0,0.0,0.0,0.0,1.0
2,0,2021-09-01 00:00:00+00:00,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325,342.0,0.0,0.0,0.0,1.0
3,0,2021-09-01 00:00:00+00:00,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325,342.0,0.0,0.0,0.0,1.0
4,0,2021-09-01 00:00:00+00:00,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325,342.0,0.0,0.0,0.0,1.0


In [31]:
# extract the hour from the 'datetime' column and create a new 'hour' column in hw_df_datetime
hw_df_datetime['hour'] = hw_df_datetime['datetime'].dt.hour
# extract the hour from the 'datetime' column and create a new 'hour' column
hw_df_datetime_county['hour'] = hw_df_datetime_county['datetime'].dt.hour
# remove duplicate rows, if any, based on all columns
hw_df_datetime.drop_duplicates(inplace=True)
# remove duplicate rows, if any, based on all columns
hw_df_datetime_county.drop_duplicates(inplace=True)
# drop the 'datetime' column as it's no longer needed after extracting hour
hw_df_datetime.drop('datetime', axis=1, inplace=True)
# drop the 'datetime' column as it's no longer needed after extracting hour
hw_df_datetime_county.drop('datetime', axis=1, inplace=True)

In [32]:
gas_df.head()

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5


In [33]:
def preTrain(data, client, hist_weather, forecast_weather, electricity, gas, locations):

    ################## data
    # drop missing values of target 
    data.dropna(subset=['target'], inplace=True)

    # convert datetime to UTC
    data['datetime'] = pd.to_datetime(data['datetime'], utc=True)

    # add year column in train data
    data['year'] = data['datetime'].dt.year

    # add month column
    data['month'] = data['datetime'].dt.month
    
    # adding day column
    data['day'] = data['datetime'].dt.day

    # add hour column
    data['hour'] = data['datetime'].dt.hour

    # add day of week column
    data['dayofweek'] = data['datetime'].dt.dayofweek

    # add day of year column
    data['dayofyear'] = data['datetime'].dt.dayofyear


    ################## electricity
    # rename 'forecast_date' column to 'datetime' for consistency before merging
    electricity = electricity.rename(columns={'forecast_date': 'datetime'}) 

    # convert datetime to UTC
    electricity['datetime'] = pd.to_datetime(electricity['datetime'], utc=True)

    # add hour column
    electricity['hour'] = electricity['datetime'].dt.hour


    ################## client
    # reduce 'block_id' in the 'client' DataFrame by 2 to match 'train' DataFrame
    client['data_block_id'] -= 2


    ################## location
    # remove the 'Unnamed: 0' column from the 'locations' DataFrame
    locations.drop('Unnamed: 0', axis=1, inplace=True)


    ################## forecast_weather
    # round 'latitude' and 'longitude' to 1 decimal place
    forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)

    # merge 'forecast_weather' with 'locations' on coordinates to add county information
    forecast_weather = forecast_weather.merge(locations, how='left', on=['longitude', 'latitude'])

    # drop missing values
    forecast_weather.dropna(axis=0, inplace=True)

    # convert the 'county' column to integer data type
    forecast_weather['county'] = forecast_weather['county'].astype('int64')

    # drop the specified columns as they are not needed for further analysis
    forecast_weather.drop(['origin_datetime', 'latitude', 'longitude', 'hours_ahead', 'data_block_id'], axis=1, inplace=True)

    # rename the 'forecast_datetime' column to 'datetime' for consistency with other datasets
    forecast_weather.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

    # convert the 'datetime' column to UTC
    forecast_weather['datetime'] = pd.to_datetime(forecast_weather['datetime'], utc=True)


    ################## forecast_weather_datetime
    # 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
    hour_period = forecast_weather['datetime'].dt.to_period('h')

    # 2. drop the columns we don't want to include in the mean calculation
    columns_to_keep = forecast_weather.drop(['county', 'datetime'], axis=1).columns

    # 3. group by the hourly period and calculate the mean for the remaining columns
    grouped = forecast_weather.groupby(hour_period)[columns_to_keep].mean()

    # 4. reset the index so we can have the hourly periods as a regular column (add an index column)
    forecast_weather_datetime = grouped.reset_index()

    # convert hourly periods back to timestamps
    # forecast_weather_datetime contains the average values for each hour, across all dates
    forecast_weather_datetime['datetime'] = pd.to_datetime(forecast_weather_datetime['datetime'].dt.to_timestamp(), utc=True)


    ################## forecast_weather_datetime_county
    # 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
    hour_period = forecast_weather['datetime'].dt.to_period('h')

    # 2. drop the columns we don't want to include in the mean calculation
    columns_to_keep = forecast_weather.drop(['county', 'datetime'], axis=1).columns

    # 3. group by the hourly period and calculate the mean for the remaining columns
    grouped = forecast_weather.groupby([hour_period, 'county'])[columns_to_keep].mean()

    # 4. reset the index so we can have the hourly periods as a regular column (add an index column)
    forecast_weather_datetime_county = grouped.reset_index()

    # convert hourly periods back to timestamps
    # forecast_weather_datetime contains the average values for each county per hour, across all dates
    forecast_weather_datetime_county['datetime'] = pd.to_datetime(forecast_weather_datetime_county['datetime'].dt.to_timestamp(), utc=True)


    ################## hist_weather
    # round 'latitude' and 'longitude' to 1 decimal place
    hist_weather[['latitude', 'longitude']] = hist_weather[['latitude', 'longitude']].astype(float).round(1)

    # add county information to 'hist_weather' based on matching coordinates
    hist_weather = hist_weather.merge(locations, how='left', on=['longitude', 'latitude'])

    # remove rows with any missing values
    hist_weather.dropna(axis=0, inplace=True)

    # remove 'latitude' and 'longitude' columns as they are no longer needed
    hist_weather.drop(['latitude', 'longitude'], axis=1, inplace=True)

    # change the 'county' column to integer datatype
    hist_weather['county'] = hist_weather['county'].astype('int64')

    # convert 'datetime' column to UTC
    hist_weather['datetime'] = pd.to_datetime(hist_weather['datetime'], utc=True)


    ################## hist_weather_datetime
    # 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
    hour_period = hist_weather['datetime'].dt.to_period('h')

    # 2. drop the columns we don't want to include in the mean calculation
    columns_to_keep = hist_weather.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

    # 3. group by the hourly period and calculate the mean for the remaining columns
    grouped = hist_weather.groupby(hour_period)[columns_to_keep].mean()

    # 4. reset the index so we can have the hourly periods as a regular column
    hist_weather_datetime = grouped.reset_index()

    # convert hourly periods back to timestamps
    # hw_df_datetime contains the average values for each hour, across all dates
    hist_weather_datetime['datetime'] = pd.to_datetime(hist_weather_datetime['datetime'].dt.to_timestamp(), utc=True)
    hist_weather_datetime = hist_weather_datetime.merge(hist_weather[['datetime', 'data_block_id']], how='left', on='datetime')


    ################## hist_weather_datetime_county
    # 1. convert the 'datetime' column to a period indexd by hour and assign it to a new variable
    hour_period = hist_weather['datetime'].dt.to_period('h')

    # 2. drop the columns we don't want to include in the mean calculation
    columns_to_keep = hist_weather.drop(['county', 'datetime', 'data_block_id'], axis=1).columns

    # 3. group by the hourly period and calculate the mean for the remaining columns
    grouped = hist_weather.groupby(['county', hour_period])[columns_to_keep].mean()

    # 4. reset the index so we can have the hourly periods as a regular column
    hist_weather_datetime_county = grouped.reset_index()

    # convert hourly periods back to timestamps
    # hw_df_datetime contains the average values for each hour, across all dates
    hist_weather_datetime_county['datetime'] = pd.to_datetime(hist_weather_datetime_county['datetime'].dt.to_timestamp(), utc=True)
    hist_weather_datetime_county = hist_weather_datetime_county.merge(hist_weather[['datetime', 'data_block_id']], how='left', on='datetime')

    # extract the hour from the 'datetime' column and create a new 'hour' column in hw_df_datetime
    hist_weather_datetime['hour'] = hist_weather_datetime['datetime'].dt.hour
    # extract the hour from the 'datetime' column and create a new 'hour' column
    hist_weather_datetime_county['hour'] = hist_weather_datetime_county['datetime'].dt.hour
    # remove duplicate rows, if any, based on all columns
    hist_weather_datetime.drop_duplicates(inplace=True)
    # remove duplicate rows, if any, based on all columns
    hist_weather_datetime_county.drop_duplicates(inplace=True)
    # drop the 'datetime' column as it's no longer needed after extracting hour
    hist_weather_datetime.drop('datetime', axis=1, inplace=True)
    # drop the 'datetime' column as it's no longer needed after extracting hour
    hist_weather_datetime_county.drop('datetime', axis=1, inplace=True)

    ################## merge train data with client data, gas data, electricity data, and ect...
    data = data.merge(client.drop(columns = ['date']), how='left', on=['data_block_id', 'county', 'is_business', 'product_type'])
    data = data.merge(gas[['data_block_id', 'lowest_price_per_mwh', 'highest_price_per_mwh']], how='left', on='data_block_id')
    data = data.merge(electricity[['euros_per_mwh', 'hour', 'data_block_id']], how='left', on=['hour', 'data_block_id'])                  
    data = data.merge(forecast_weather_datetime, how='left', on=['datetime'])                  
    data = data.merge(forecast_weather_datetime_county, how='left', on=['datetime', 'county'], suffixes=('_fcast_mean', '_fcast_mean_by_county'))      
    data = data.merge(hist_weather_datetime, how='left', on=['data_block_id', 'hour'])
    data = data.merge(hist_weather_datetime_county, how='left', on=['data_block_id', 'county', 'hour'], suffixes=('_hist_mean', '_hist_mean_by_county'))
    data = data.groupby(['year', 'day', 'hour'], as_index=False).apply(lambda x: x.ffill().bfill().reset_index())
    data.drop(['row_id', 'data_block_id', 'year', 'index'], axis=1, inplace=True)
    
    return data                      

In [34]:
def create_revealed_targets_train(data, N_day_lags):
    '''Creating lagged target values | A target value depends on what was a target value n days ago'''
    
    # keep original datetime for reference
    original_datetime = data['datetime']
    
    # select columns for merging lagged targets
    revealed_targets = data[['datetime', 'prediction_unit_id', 'is_consumption', 'target']].copy()
    
    # loop to create and merge lagged targets
    for day_lag in range(2, N_day_lags+1):
        # Shift datetime for the lag
        revealed_targets['datetime'] = original_datetime + pd.DateOffset(day_lag)
        # Merge lagged targets into data
        data = data.merge(revealed_targets, 
                          how='left', 
                          on=['datetime', 'prediction_unit_id', 'is_consumption'],
                          suffixes=('', f'_{day_lag}_days_ago'))
    return data

In [35]:
#### read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
gas_df = pd.read_csv(os.path.join(DATA_DIR, "gas_prices.csv"))
electricity_df = pd.read_csv(os.path.join(DATA_DIR, "electricity_prices.csv"))
client_df = pd.read_csv(os.path.join(DATA_DIR, "client.csv"))
fw_df = pd.read_csv(os.path.join(DATA_DIR, "forecast_weather.csv"))
hw_df = pd.read_csv(os.path.join(DATA_DIR, "historical_weather.csv"))

# read a file from a different directory
# see how this data is generated: https://www.kaggle.com/jackren000/mapping-locations-and-county-code/edit
locations = pd.read_csv("/kaggle/input/county-lon-lats/county_lon_lats.csv")

In [36]:
# apply the train function
train = preTrain(train, client_df, hw_df, fw_df, electricity_df, gas_df, locations)

# specify how many days to lag and apply the function
N_day_lags = 7
train = create_revealed_targets_train(train, N_day_lags = N_day_lags)

# convert the datetime to integer
train['datetime'] = train['datetime'].astype('int64')

In [37]:
print("Train shape is " + str(train.shape))
train.head()

Train shape is (2017824, 75)


,county,is_business,product_type,target,is_consumption,datetime,prediction_unit_id,month,day,hour,...,winddirection_10m_hist_mean_by_county,shortwave_radiation_hist_mean_by_county,direct_solar_radiation_hist_mean_by_county,diffuse_radiation_hist_mean_by_county,target_2_days_ago,target_3_days_ago,target_4_days_ago,target_5_days_ago,target_6_days_ago,target_7_days_ago
0,0,0,1,0.713,0,1630454400000000000,0,9,1,0,...,143.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,1630454400000000000,0,9,1,0,...,143.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,1630454400000000000,1,9,1,0,...,143.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,1630454400000000000,1,9,1,0,...,143.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,1630454400000000000,2,9,1,0,...,143.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017824 entries, 0 to 2017823
Data columns (total 75 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   county                                                  int64  
 1   is_business                                             int64  
 2   product_type                                            int64  
 3   target                                                  float64
 4   is_consumption                                          int64  
 5   datetime                                                int64  
 6   prediction_unit_id                                      int64  
 7   month                                                   int32  
 8   day                                                     int32  
 9   hour                                                    int32  
 10  dayofweek                                             

### Data Transformation

In [39]:
train['hour'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int32)

In [40]:
train['dayofyear'].unique()

array([244, 274, 305, 335, 245, 275, 306, 336, 246, 276, 307, 337, 247,
       277, 308, 338, 248, 278, 309, 339, 249, 279, 310, 340, 250, 280,
       311, 341, 251, 281, 312, 342, 252, 282, 313, 343, 253, 283, 314,
       344, 254, 284, 315, 345, 255, 285, 316, 346, 256, 286, 317, 347,
       257, 287, 318, 348, 258, 288, 319, 349, 259, 289, 320, 350, 260,
       290, 321, 351, 261, 291, 322, 352, 262, 292, 323, 353, 263, 293,
       324, 354, 264, 294, 325, 355, 265, 295, 326, 356, 266, 296, 327,
       357, 267, 297, 328, 358, 268, 298, 329, 359, 269, 299, 330, 360,
       270, 300, 331, 361, 271, 301, 332, 362, 272, 302, 333, 363, 273,
       303, 334, 364, 304, 365,   1,  32,  60,  91, 121, 152, 182, 213,
         2,  33,  61,  92, 122, 153, 183, 214,   3,  34,  62,  93, 123,
       154, 184, 215,   4,  35,  63,  94, 124, 155, 185, 216,   5,  36,
        64,  95, 125, 156, 186, 217,   6,  37,  65,  96, 126, 157, 187,
       218,   7,  38,  66,  97, 127, 158, 188, 219,   8,  39,  6

In [41]:
#### ceate features
# normalize and convert 'hour' to sine values (24-hour cycle)
train['sin_hour'] = np.sin((train['hour'] / 24.0) * (2 * np.pi))

# normalize and convert 'hour' to cosine values (24-hour cycle)
train['cos_hour'] = np.cos((train['hour'] / 24.0) * (2 * np.pi))

# normalize and convert 'dayofyear' to sine values (annual cycle)
train['sin_dayofyear'] = np.sin((train['dayofyear'] / 365.0) * (2 * np.pi))

# normalize and convert 'dayofyear' to cosine values (annual cycle)
train['cos_dayofyear'] = np.cos((train['dayofyear'] / 365.0) * (2 * np.pi))

# calculate mean of historical target values
N_day_lags = 7
train['target_mean'] = train[[f'target_{i}_days_ago' for i in range(2, N_day_lags+1)]].mean(axis=1)

# calculate standard deviation of historical target values
train['target_std'] = train[[f'target_{i}_days_ago' for i in range(2, N_day_lags+1)]].std(axis=1)

# calculate variance of historical target values
train['target_var'] = train[[f'target_{i}_days_ago' for i in range(2, N_day_lags+1)]].var(axis=1)

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017824 entries, 0 to 2017823
Data columns (total 82 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   county                                                  int64  
 1   is_business                                             int64  
 2   product_type                                            int64  
 3   target                                                  float64
 4   is_consumption                                          int64  
 5   datetime                                                int64  
 6   prediction_unit_id                                      int64  
 7   month                                                   int32  
 8   day                                                     int32  
 9   hour                                                    int32  
 10  dayofweek                                             

In [43]:
#Log columns with outliers
to_log= ['installed_capacity', 'euros_per_mwh', 'temperature_fcast_mean', 'dewpoint_fcast_mean',
        'cloudcover_high_fcast_mean', 'cloudcover_low_fcast_mean', 'cloudcover_mid_fcast_mean', 'cloudcover_total_fcast_mean',
        '10_metre_u_wind_component_fcast_mean', '10_metre_v_wind_component_fcast_mean', 'direct_solar_radiation_fcast_mean',
        'snowfall_fcast_mean', 'total_precipitation_fcast_mean', 'temperature_fcast_mean_by_county', 'dewpoint_fcast_mean_by_county',
        'cloudcover_high_fcast_mean_by_county', 'cloudcover_low_fcast_mean_by_county', 'cloudcover_mid_fcast_mean_by_county',
        'cloudcover_total_fcast_mean_by_county', '10_metre_u_wind_component_fcast_mean_by_county', '10_metre_v_wind_component_fcast_mean_by_county',
        'surface_solar_radiation_downwards_fcast_mean_by_county', 'snowfall_fcast_mean_by_county', 'total_precipitation_fcast_mean_by_county',
        'rain_hist_mean', 'snowfall_hist_mean', 'windspeed_10m_hist_mean_by_county', 'target_2_days_ago', 'target_3_days_ago',
        'target_4_days_ago', 'target_5_days_ago', 'target_6_days_ago', 'target_7_days_ago', 'target_mean', 'target_std']
for i in to_log:
    train[f"log_{i}"]= np.where((train[i])!= 0, np.log(train[i]),0)

In [44]:
train.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017824 entries, 0 to 2017823
Data columns (total 117 columns):
 #    Column                                                      Dtype  
---   ------                                                      -----  
 0    county                                                      int64  
 1    is_business                                                 int64  
 2    product_type                                                int64  
 3    target                                                      float64
 4    is_consumption                                              int64  
 5    datetime                                                    int64  
 6    prediction_unit_id                                          int64  
 7    month                                                       int32  
 8    day                                                         int32  
 9    hour                                                        int32 

In [45]:
# save the processed data
train.to_csv('processed_train.csv', index=False)

#### Continue with the construction of the model
following the link: [LSTM Predict Energy Behavior of Prosumers on Kaggle](https://www.kaggle.com/code/jackren000/lstm-predict-energy-behavior-of-prosumers).